<a href="https://colab.research.google.com/github/HunterCLcode/t5fakereview/blob/main/t5FakeReviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The goal of this t5FakeReviews is creating Fake Reviews from just the product name or category or both.

Imports & Version Control

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import pandas as pd

Mount Drive

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Set Folder to use...
WORK_DIR = '/content/drive/MyDrive/CS 4V98 Shared Data'
os.makedirs(WORK_DIR, exist_ok = True) 